In [1]:
#数据
my_data=[['slashdot','USA','yes',18,'None'],

['google','France','yes',23,'Premium'],

['digg','USA','yes',24,'Basic'],

['kiwitobes','France','yes',23,'Basic'],

['google','UK','no',21,'Premium'],

['(direct)','New Zealand','no',12,'None'],

['(direct)','UK','no',21,'Basic'],

['google','USA','no',24,'Premium'],

['slashdot','France','yes',19,'None'],

['digg','USA','no',18,'None'],

['google','UK','no',18,'None'],

['kiwitobes','UK','no',19,'None'],

['digg','New Zealand','yes',12,'Basic'],

['slashdot','UK','no',21,'None'],

['google','UK','yes',18,'Basic'],

['kiwitobes','France','yes',19,'Basic']]

In [2]:
import pandas as pd
import numpy as np


In [3]:
data = pd.DataFrame(my_data)

In [4]:
data_targets = data[data.columns[4]]

In [5]:
data_targets

0        None
1     Premium
2       Basic
3       Basic
4     Premium
5        None
6       Basic
7     Premium
8        None
9        None
10       None
11       None
12      Basic
13       None
14      Basic
15      Basic
Name: 4, dtype: object

In [6]:
data_x = data.drop(data.columns[4],axis=1)

In [7]:
data.columns[4]

4

In [8]:
data_x

,0,1,2,3
0,slashdot,USA,yes,18
1,google,France,yes,23
2,digg,USA,yes,24
3,kiwitobes,France,yes,23
4,google,UK,no,21
5,(direct),New Zealand,no,12
6,(direct),UK,no,21
7,google,USA,no,24
8,slashdot,France,yes,19
9,digg,USA,no,18


In [9]:
data_x.columns = ['site','loc','faq','pagenum']

In [10]:
data_x

,site,loc,faq,pagenum
0,slashdot,USA,yes,18
1,google,France,yes,23
2,digg,USA,yes,24
3,kiwitobes,France,yes,23
4,google,UK,no,21
5,(direct),New Zealand,no,12
6,(direct),UK,no,21
7,google,USA,no,24
8,slashdot,France,yes,19
9,digg,USA,no,18


#数据预处理
已经得到x和targets数据
1、数值化
    one-hot
2、归一化

ROC曲线
特征之间协方差找线性相关系数

In [11]:
#[slashdot google digg kiwitobes (direct)] ->[1 2 3 4 5]
data_x.replace('slashdot',1, inplace=True)
data_x.replace('google',2, inplace=True)
data_x.replace('digg',3, inplace=True)
data_x.replace('kiwitobes',4, inplace=True)
data_x.replace('(direct)',5, inplace=True)

In [12]:
data_x

,site,loc,faq,pagenum
0,1,USA,yes,18
1,2,France,yes,23
2,3,USA,yes,24
3,4,France,yes,23
4,2,UK,no,21
5,5,New Zealand,no,12
6,5,UK,no,21
7,2,USA,no,24
8,1,France,yes,19
9,3,USA,no,18


In [13]:
#[USA France UK New Zealand ] ->[1 2 3 4 5]
data_x.replace('USA',1, inplace=True)
data_x.replace('France',2, inplace=True)
data_x.replace('UK',3, inplace=True)
data_x.replace('New Zealand',4, inplace=True)

In [14]:
data_x

,site,loc,faq,pagenum
0,1,1,yes,18
1,2,2,yes,23
2,3,1,yes,24
3,4,2,yes,23
4,2,3,no,21
5,5,4,no,12
6,5,3,no,21
7,2,1,no,24
8,1,2,yes,19
9,3,1,no,18


In [15]:
#[yes no ] ->[0 1]
data_x.replace('yes',1, inplace=True)
data_x.replace('no',0, inplace=True)

In [16]:
data_x

,site,loc,faq,pagenum
0,1,1,1,18
1,2,2,1,23
2,3,1,1,24
3,4,2,1,23
4,2,3,0,21
5,5,4,0,12
6,5,3,0,21
7,2,1,0,24
8,1,2,1,19
9,3,1,0,18


In [17]:
#0        None
# 1     Premium
# 2       Basic
data_targets.replace('None',0, inplace=True)
data_targets.replace('Premium',1, inplace=True)
data_targets.replace('Basic',2, inplace=True)

In [18]:
data_targets

0     0
1     1
2     2
3     2
4     1
5     0
6     2
7     1
8     0
9     0
10    0
11    0
12    2
13    0
14    2
15    2
Name: 4, dtype: int64

In [19]:
from sklearn import preprocessing
#导入数据处理模块，并生成one_hot()编码函数
enc = preprocessing.OneHotEncoder()

In [20]:
enc.fit(data_x.iloc[:,0:3])

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [25]:
enc.n_values_

array([6, 5, 2])

In [22]:
x = enc.transform(data_x.iloc[:,0:3]).toarray()

In [28]:
x = pd.DataFrame(x)

In [34]:
#归一化
from sklearn import preprocessing

In [37]:
min_max_scaler = preprocessing.MinMaxScaler()
add = min_max_scaler.fit_transform(data_x['pagenum'].reshape(-1,1))

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [38]:
add

array([[0.5       ],
       [0.91666667],
       [1.        ],
       [0.91666667],
       [0.75      ],
       [0.        ],
       [0.75      ],
       [1.        ],
       [0.58333333],
       [0.5       ],
       [0.5       ],
       [0.58333333],
       [0.        ],
       [0.75      ],
       [0.5       ],
       [0.58333333]])

In [39]:
x[11] = add

In [40]:
x

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.500000
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.916667
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.000000
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.916667
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.750000
5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.000000
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.750000
7,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.000000
8,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.583333
9,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.500000


In [57]:
from sklearn.cross_validation import train_test_split

In [60]:
#拆分训练数据与测试数据 
x_train, x_test, y_train, y_test = train_test_split(x,data_targets, test_size = 0.2)

In [66]:
####贝叶斯分类器####
#训练贝叶斯分类器
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB() 
clf.fit(x_train,y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [47]:
from sklearn.metrics import classification_report

In [67]:
#测试结果
answer = clf.predict(x_test)
print(x_test)
print(answer)
print(y_test)
print(np.mean( answer == y_test))


     0    1    2    3    4    5    6    7    8    9    10   11
14  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.5
9   0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.5
5   0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0
12  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  0.0
[1 0 2 2]
14    2
9     0
5     0
12    2
Name: 4, dtype: int64
0.5


In [63]:
print(np.mean( answer == y_test))

0.5
